# Shape classification using CNN

In this notebook, we will be using the model to perform predictions


In [2]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import numpy as np
from torch.autograd import Variable
from torchvision.models import squeezenet1_1
import torch.functional as F
from io import open
import os
from PIL import Image
from pathlib import Path
import glob
import cv2

## loading the data

This cell loads the data to be predicted from the pred folder in the datasets folder

In [3]:
train = Path.cwd() / "dataset"/"train"
pred = Path.cwd() / "dataset"/"pred"

In [276]:
#categories
root=Path(train)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)

['circle', 'square', 'triangle']


## Defining the architecture 

In [277]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=4):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=100 * 100 * 32,out_features=num_classes)
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
        
        
        #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*100*100)
            
            
        output=self.fc(output)
            
        return output

## Loading the saved model

In [278]:
checkpoint=torch.load('best_checkpoint.model')
model=ConvNet(num_classes=4)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=320000, out_features=4, bias=True)
)

## Transforming the images to be predicted


In [279]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((200,200)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

## Defining the prediction fuction

In [283]:
#prediction function
def prediction(img_path,transformer):
    
    image=Image.open(img_path)
    
    
    image_tensor=transformer(image).float()
    
    
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensor)
    
    
    output=model(input)
    
    index=output.data.numpy().argmax()
    
    pred=classes[index]
    
    return pred

## Performing the predictions

In [284]:
images_path=glob.glob(str(pred)+'/*.png')

In [285]:
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)

## Prediction Results

In [286]:
pred_dict

{'C:\\Users\\Sizamkele Ngqwemla\\Documents\\Mechatronics_Engineering\\4th_Year_Academics\\First_Semester\\EEE4114F_DSP\\project2\\dataset\\pred\\C1.png': 'circle',
 'C:\\Users\\Sizamkele Ngqwemla\\Documents\\Mechatronics_Engineering\\4th_Year_Academics\\First_Semester\\EEE4114F_DSP\\project2\\dataset\\pred\\C2.png': 'circle',
 'C:\\Users\\Sizamkele Ngqwemla\\Documents\\Mechatronics_Engineering\\4th_Year_Academics\\First_Semester\\EEE4114F_DSP\\project2\\dataset\\pred\\S1.png': 'square',
 'C:\\Users\\Sizamkele Ngqwemla\\Documents\\Mechatronics_Engineering\\4th_Year_Academics\\First_Semester\\EEE4114F_DSP\\project2\\dataset\\pred\\S2.png': 'triangle',
 'C:\\Users\\Sizamkele Ngqwemla\\Documents\\Mechatronics_Engineering\\4th_Year_Academics\\First_Semester\\EEE4114F_DSP\\project2\\dataset\\pred\\T1.png': 'triangle',
 'C:\\Users\\Sizamkele Ngqwemla\\Documents\\Mechatronics_Engineering\\4th_Year_Academics\\First_Semester\\EEE4114F_DSP\\project2\\dataset\\pred\\T3.png': 'triangle'}